In [6]:
import numpy as np
import re

In [8]:
Rs = [0.00012646895,0.00030986862,0.0090425299,8.4991556e-05]
Rb = [6.3234475e-05,4.4507635e-05,0.00052695395,4.2466678e-05]
As = [1.69445,3.38889,6.29366,0.968255]
Ab = [26.8691,13.5556,41.877,21.3016]
Cs = [6,14,429,4]
Cb = [3,2,25,2]
Ts = [47442.5,45147.2,47442.5,47078.9]
Tb = [47442.5,44913.3,47442.1,47073.4]

In [10]:
Rs = np.array(Rs)
Rb = np.array(Rb)
As = np.array(As)
Ab = np.array(Ab)
Cs = np.array(Cs)
Cb = np.array(Cb)
Ts = np.array(Ts)
Tb = np.array(Tb)

In [12]:
Rnet = Rs - Rb*(As/Ab)

sigma_Rs = np.sqrt(Cs) / Ts
sigma_Rb = np.sqrt(Cb) / Tb

sigma_Rnet = np.sqrt(sigma_Rs**2 + (sigma_Rb * As / Ab)**2)

In [14]:
SNR = Rnet/sigma_Rnet

In [16]:
SNR

array([ 2.36989965,  3.58849586, 20.51742638,  1.9542068 ])

In [18]:
absorved = 4.162E-12 
unabsorved = 5.185E-12  

flux_abs = Rnet * absorved
flux_abs_err = sigma_Rnet * absorved
flux_unabs = Rnet * unabsorved
flux_unabs_err = sigma_Rnet * unabsorved

In [20]:
def luminosidad(flux, distance, flux_error, distance_error):
    # Convertir distancia a cm (si es necesario)
    distance = distance * 3.086 * 10**24
    
    L = 4 * np.pi * distance**2 * flux
    L_error = np.sqrt((4 * np.pi * distance**2 * flux_error)**2 + (8 * np.pi * distance * flux * distance_error)**2)
    
    return L, L_error

dist =  5.16 # en Mpc
e_dist = 0.35

lumi_abs, lumi_abs_err = luminosidad(flux_abs, dist, flux_abs_err, e_dist)
lumi_unabs, lumi_unabs_err = luminosidad(flux_unabs, dist, flux_unabs_err, e_dist)

In [22]:
def cargar_coords(nombre_reg):
    x_list = []
    y_list = []
    r_list = []

    with open(nombre_reg, 'r') as f:
        for line in f:
            if line.startswith("circle("):
                # Cerca valors dins de circle(...)
                match = re.search(r'circle\(([^,]+),([^,]+),([^\)"]+)', line)
                if match:
                    x, y, r = map(float, match.groups())
                    x_list.append(x)
                    y_list.append(y)
                    r_list.append(r)
    
    return np.array(x_list), np.array(y_list), np.array(r_list)

In [58]:
path = '15385/wavdetect/broad_results_area_WCS.reg'
coords = cargar_coords(path)

In [60]:
x,y,r=coords

In [54]:
import csv

energy = [['hard','medium','soft'],['hard','medium','soft'],['hard','medium','soft'],['medium','soft']]

header = [
    'RA', 'Dec',
    '$F_{abs}$', '$F_{unabs}$',
    'flux_abs_err', 'flux_unabs_err',
    'log($L_{abs}$)', 'log10($L_{unabs}$)',
    'lumi_abs_err', 'lumi_unabs_err', 'Broad S/N', 'Detected in bands'
]

log_lumi_unabs = np.log10(lumi_unabs)
log_lumi_abs = np.log10(lumi_abs)

log_lumi_unabs_err = np.log10(lumi_unabs + lumi_unabs_err) - log_lumi_unabs
log_lumi_abs_err = np.log10(lumi_abs + lumi_abs_err) - log_lumi_abs


rows = zip(
    x, y,
    flux_abs, flux_unabs,
    flux_abs_err, flux_unabs_err,
    log_lumi_abs, log_lumi_unabs,
    log_lumi_abs_err, log_lumi_unabs_err, SNR, energy
)

# Guardem a fitxer CSV
with open('resultats_xray_NGC5068.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)


In [42]:
print(log_lumi_abs_err)

[0.15288701 0.10675789 0.02066744 0.17947037]
